In [3]:
import pandas as pd
data = pd.read_csv('RFM201810.csv',low_memory = False)

In [4]:
data

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2018-10-01 00:00:00.000,1716.00
1,C0011810010017,13233933,2018-10-01 00:00:00.000,1489.74
2,C0011810010020,99057968,2018-10-01 00:00:00.000,151.47
3,C0011810010021,80007276,2018-10-01 00:00:00.000,146.72
4,C0011810010024,13164076,2018-10-01 00:00:00.000,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2018-10-31 00:00:00.000,3801.87
332726,S0081810310461,99065678,2018-10-31 00:00:00.000,5769.88
332727,S0081810310462,19029918,2018-10-31 00:00:00.000,736.88
332728,S0081810310463,13020033,2018-10-31 00:00:00.000,1475.20


In [5]:
import datetime as dt
NOW = dt.datetime(2018,10,31)

In [6]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])


In [14]:
rfmtable = data.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),# Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value Общая сумма по всем заказам



rfmtable['InvoiceDate'] = rfmtable['InvoiceDate'].astype(int)
rfmtable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [19]:
quantiles = rfmtable.quantile(q=[0.25,0.5,0.75])

In [20]:
rfmSegmentation = rfmtable

In [21]:
# R-Recensy
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# F-Frequency, M-Monetary Value
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [22]:
#преобразование таблицы
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

rfmSegmentation['RFMClass'].value_counts()

444    10603
111     9712
443     6726
344     6578
211     5853
       ...  
424       63
114       60
214       60
314       33
414        2
Name: RFMClass, Length: 64, dtype: int64